<a href="https://colab.research.google.com/github/AlexOchoki/Investigate-a-Dataset-TMDb-Movies-Dataset/blob/main/update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Install (if needed) and import libraries for Excel file handling
!pip install --quiet openpyxl xlrd

# Core libraries
import pandas as pd
import numpy as np
import re
import os
from pathlib import Path
# Excel workbook operations (optional: only if you modify Excel structure like formatting, formulas)
from openpyxl import load_workbook


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
# Define base folders
base_dir = Path('/content/drive/MyDrive/SNS/Ethiopia/2025/aug/dirty')
clean_dir = Path('/content/drive/MyDrive/SNS/Ethiopia/2025/aug/clean')
constants_dir = Path('/content/drive/MyDrive/SNS/Ethiopia/eth_constants')

# File paths
dirty_file = base_dir / 'aug_dirty.xlsx'
previous_month_file = base_dir / 'clean_maxerience_jul.xlsx'
ired_file = base_dir / 'iRED_MD_jul.xlsx'
latest_coordinates_file = constants_dir / 'coordinates.xlsx'

# Mapping file and its sheets
mapping_file = constants_dir / 'eth_mappings.xlsx'

In [5]:


# Load main data
dirty_df = pd.read_excel(dirty_file)
previous_month_df = pd.read_excel(previous_month_file)
ired_df = pd.read_excel(ired_file)
latest_coordinates_df = pd.read_excel(latest_coordinates_file)

# Load mapping sheets
sheet_names = {
    'picos_regions': 'picos_regions',
    'subchannel_map': 'subchannel_to_trade_channel',
    'users': 'new_user_credentials',
    'rsm': 'rsm_map',
    'asm': 'asm_map',
    'csr': 'csr_map',
    'week_dates': 'week_dates_map'
}

picos_regions_df = pd.read_excel(mapping_file, sheet_name=sheet_names['picos_regions'])
subchannel_map_df = pd.read_excel(mapping_file, sheet_name=sheet_names['subchannel_map'])
users_df = pd.read_excel(mapping_file, sheet_name=sheet_names['users'])
rsm_df = pd.read_excel(mapping_file, sheet_name=sheet_names['rsm'])
asm_df = pd.read_excel(mapping_file, sheet_name=sheet_names['asm'])
csr_df = pd.read_excel(mapping_file, sheet_name=sheet_names['csr'])
week_dates_df = pd.read_excel(mapping_file, sheet_name=sheet_names['week_dates'])


In [6]:
# Define base folders
base_dir = Path('/content/drive/MyDrive/SNS/Ethiopia/2025/aug/dirty')
constants_dir = Path('/content/drive/MyDrive/SNS/Ethiopia/eth_constants')

# Define file paths
dirty_file = base_dir / 'aug_dirty.xlsx'
previous_month_file = base_dir / 'clean_maxerience_jul.xlsx'
ired_file = base_dir / 'iRED_MD_jul.xlsx'
latest_coordinates_file = constants_dir / 'coordinates.xlsx'
mapping_file = constants_dir / 'eth_mappings.xlsx'

# Load main data
dirty_df = pd.read_excel(dirty_file)
previous_month_df = pd.read_excel(previous_month_file)
ired_df = pd.read_excel(ired_file)
latest_coordinates_df = pd.read_excel(latest_coordinates_file)

# Load mapping sheets (your clean version)
picos_regions_df = pd.read_excel(mapping_file, sheet_name='picos_regions')
subchannel_map_df = pd.read_excel(mapping_file, sheet_name='subchannel_to_trade_channel')
users_df = pd.read_excel(mapping_file, sheet_name='new_user_credentials')
rsm_df = pd.read_excel(mapping_file, sheet_name='rsm_map')
asm_df = pd.read_excel(mapping_file, sheet_name='asm_map')
csr_df = pd.read_excel(mapping_file, sheet_name='csr_map')
week_dates_df = pd.read_excel(mapping_file, sheet_name='week_dates_map')


In [8]:
# Set pandas display options to show all columns and prevent column truncation
pd.set_option('display.max_columns', None)      # Show all columns
pd.set_option('display.max_rows', 100)          # Optional: control max rows in interactive output
pd.set_option('display.max_colwidth', None)     # Show full column content without truncation
# pd.set_option('display.width', 1000)          # Uncomment if you want wide-format printing in terminals

# Preview first 5 rows of raw data
dirty_df.head(5)


,RSM Ref.,ASM Ref.,CSR Ref.,Outlet SAP,MSD Code,Sales Territory,Region,DISTRICT,City,RSM Name,ASM Name,OCCD Code,OCCD NAME,CSR Name,CSR SAP,Surv. No.,Week day,rout Status,Sub route Number,Call Day,EDS Code,Customer Name,Owner name,Land Mark,Alternative GPS,Y - Coordinat,X _ Coordinat,Phone number,Channel Code,Sub Channel,Main Channel,Sub Channel.1,Coke Cooler Availability,Cooler Type,Cooler Serial number,Cooler asset Number,KO Weekly RGB VPO,KO Weekly PET VPO,4.Ambo Weekly RGB VPO,Ambo Weekly PET VPO,RGB weekly VPO,PET weekly VPO,Total Weekly,RGB Monthly VPO,PET Monthly VPO,Monthly VPO,Leveling
0,RSM 1,ASM 27,CSR 50,7777912,11275504,Addis Ababa,SOUTH & WEST ADDIS,SOUTH ADDIS,Addis City,MOHAMMED AHMED,HANA ZERFU,7006341.0,LIDIYA GORFU,ABEL G/MICHAEL,74925,13,4.0,RED,14.0,2,TRR1305,Kebe restaurant,buruk,58,8.9354 38.747,8.9354,38.747,923792779,151 Fine restaurant,HORECA/CAFE,ON-PREMISES,HORECA/CAFE,Cooler,Cooler,ET00123744,No Asset,4,0,3.0,0.0,7.0,0.0,7.0,30.31,0.0,30.31,BRONZ
1,RSM 1,ASM 27,CSR 50,7773991,11271759,Addis Ababa,SOUTH & WEST ADDIS,SOUTH ADDIS,Addis City,MOHAMMED AHMED,HANA ZERFU,7006341.0,LIDIYA GORFU,ABEL G/MICHAEL,74925,13,4.0,RED,14.0,2,TRR1502,Mame Grossery,Abreham kebru,58,8.9374 38.7435,8.9374,38.7435,912724119,200 Grocery (Bar),BUTCHERY,ON-PREMISES,BUTCHERY,None Cooler,None Cooler,None Cooler,None Cooler,4,0,1.0,0.0,5.0,0.0,5.0,21.65,0.0,21.65,BRONZ
2,RSM 1,ASM 27,CSR 50,7773990,11271758,Addis Ababa,SOUTH & WEST ADDIS,SOUTH ADDIS,Addis City,MOHAMMED AHMED,HANA ZERFU,7006341.0,LIDIYA GORFU,ABEL G/MICHAEL,74925,13,4.0,RED,16.0,4,TRR1501,Kedus bar and restaurant,Teruwerk lengerew,hana kelebet,8.9367 38.7438,8.9367,38.7438,911666873,151 Fine restaurant,HORECA/CAFE,ON-PREMISES,HORECA/CAFE,None Cooler,None Cooler,None Cooler,None Cooler,2,0,0.0,0.0,2.0,0.0,2.0,8.66,0.0,8.66,BRONZ
3,RSM 1,ASM 27,CSR 50,7773977,11271745,Addis Ababa,SOUTH & WEST ADDIS,SOUTH ADDIS,Addis City,MOHAMMED AHMED,HANA ZERFU,7006341.0,LIDIYA GORFU,ABEL G/MICHAEL,74925,13,4.0,RED,14.0,2,TRR1421,Yosef makefafeya,yosef birhanu,58,8.9371 38.7456,8.9371,38.7456,911807227,351 Beverage wholesaler,LIQUOR,LIQUOR (LIQUOR STORE),LIQUOR,None Cooler,None Cooler,None Cooler,None Cooler,4,0,1.0,0.0,5.0,0.0,5.0,21.65,0.0,21.65,BRONZ
4,RSM 1,ASM 27,CSR 50,7773967,11271735,Addis Ababa,SOUTH & WEST ADDIS,SOUTH ADDIS,Addis City,MOHAMMED AHMED,HANA ZERFU,7006341.0,LIDIYA GORFU,ABEL G/MICHAEL,74925,13,4.0,RED,14.0,2,TRR1414,Dagim hotel,tewabch tamne,58,8.9389 38.7475,8.9389,38.7475,911359932,401 Hotel,HORECA/CAFE,ON-PREMISES,HORECA/CAFE,None Cooler,None Cooler,None Cooler,None Cooler,1,0,1.0,0.0,2.0,0.0,2.0,8.66,0.0,8.66,BRONZ


In [11]:
print("===== dirty_df Exploration =====\n")

# 1. Shape of the DataFrame
print(f"Shape (Rows, Columns): {dirty_df.shape}")

# 2. Missing values: count and percentage
missing_count = dirty_df.isnull().sum().astype(int)
missing_pct = (dirty_df.isnull().mean() * 100).round(2)

# 3. Build summary DataFrame
missing_summary = pd.DataFrame({
    'Missing Count': missing_count,
    'Missing %': missing_pct.map(lambda x: f"{int(x) if x.is_integer() else x}%")
})

# 4. Display summary
print("\nMissing Values Summary:")
display(missing_summary)

# 5. Save to clean folder
clean_dir.mkdir(parents=True, exist_ok=True)
output_path = clean_dir / 'missing_summary.xlsx'
missing_summary.to_excel(output_path, index=True)

print(f"\n✅ Saved missing value summary to: {output_path}")
print("\n" + "=" * 50)


===== dirty_df Exploration =====

Shape (Rows, Columns): (25537, 47)

Missing Values Summary:


,Missing Count,Missing %
RSM Ref.,0,0%
ASM Ref.,0,0%
CSR Ref.,0,0%
Outlet SAP,9547,37.38%
MSD Code,0,0%
Sales Territory,0,0%
Region,0,0%
DISTRICT,0,0%
City,297,1.16%
RSM Name,0,0%



✅ Saved missing value summary to: /content/drive/MyDrive/SNS/Ethiopia/2025/aug/clean/missing_summary.xlsx



In [14]:
# List of important columns to check
important_columns = [
    "RSM Ref.", "ASM Ref.", "CSR Ref.", "MSD Code", "RSM Name",
    "ASM Name", "CSR Name", "Surv. No.", "Week day", "Customer Name", "Leveling"
]

print("===== Key Column Null Check Summary =====\n")

total_rows = len(dirty_df)

for col in important_columns:
    null_count = dirty_df[col].isnull().sum()
    null_pct = (null_count / total_rows) * 100

    if null_count > 0:
        print(f"⚠️  '{col}': {null_count} nulls ({null_pct:.2f}%) — recheck or ask the Country Champion to reshare the file.")
    else:
        print(f"✅  '{col}' has no missing values.")

print("\n" + "=" * 50)

===== Key Column Null Check Summary =====

✅  'RSM Ref.' has no missing values.
✅  'ASM Ref.' has no missing values.
✅  'CSR Ref.' has no missing values.
✅  'MSD Code' has no missing values.
✅  'RSM Name' has no missing values.
✅  'ASM Name' has no missing values.
✅  'CSR Name' has no missing values.
✅  'Surv. No.' has no missing values.
✅  'Week day' has no missing values.
✅  'Customer Name' has no missing values.
✅  'Leveling' has no missing values.



In [13]:

# Define important columns to check for completeness
important_columns = [
    "RSM Ref.", "ASM Ref.", "CSR Ref.", "MSD Code", "RSM Name",
    "ASM Name", "CSR Name", "Surv. No.", "Week day", "Customer Name", "Leveling"
]

print("===== Key Column Null Check Summary =====\n")

rows_before = len(dirty_df)
nulls_removed = []

# Drop rows with nulls in any of the important columns, and track removals
for col in important_columns:
    null_count = dirty_df[col].isnull().sum()
    null_pct = (null_count / len(dirty_df)) * 100

    if null_count > 0:
        print(f"⚠️  '{col}': {null_count} nulls ({null_pct:.2f}%) — rows will be removed.")
        nulls_removed.append((col, null_count))
        dirty_df = dirty_df[dirty_df[col].notnull()]  # Drop rows with null in this column
    else:
        print(f"✅  '{col}' has no missing values.")

rows_after = len(dirty_df)
rows_dropped = rows_before - rows_after

# Summary
print(f"\n🧹 Total rows before cleaning: {rows_before}")
print(f"🧹 Total rows after cleaning:  {rows_after}")
print(f"🗑️  Total rows removed:        {rows_dropped}\n")

for col, count in nulls_removed:
    print(f"   • {count} rows removed due to nulls in '{col}'")

print("\n" + "=" * 50)



===== Key Column Null Check Summary =====

✅  'RSM Ref.' has no missing values.
✅  'ASM Ref.' has no missing values.
✅  'CSR Ref.' has no missing values.
✅  'MSD Code' has no missing values.
✅  'RSM Name' has no missing values.
✅  'ASM Name' has no missing values.
✅  'CSR Name' has no missing values.
✅  'Surv. No.' has no missing values.
✅  'Week day' has no missing values.
✅  'Customer Name' has no missing values.
✅  'Leveling' has no missing values.

🧹 Total rows before cleaning: 24639
🧹 Total rows after cleaning:  24639
🗑️  Total rows removed:        0


